In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

In [2]:
news_df = pd.read_csv('../data/filtered_news_with_sentiment.csv')
news_df

,Unnamed: 0,title,snippet,pub_date,web_url,section,source,year,month,title_word_count,snippet_word_count,combined_text,sentiment_label,sentiment_score
0,2,‘The New Bauhaus’ Review: Rethinking an Approa...,This documentary on the interdisciplinary arti...,2021-07-20 11:00:05,https://www.nytimes.com/2021/07/20/movies/the-...,Movies,The New York Times,2021,7,9,18,‘The New Bauhaus’ Review: Rethinking an Approa...,POSITIVE,0.998725
1,3,Apple delays its return to office as the Delta...,Employees are now expected to come back to the...,2021-07-20 15:42:40,https://www.nytimes.com/2021/07/20/technology/...,Technology,The New York Times,2021,7,11,19,Apple delays its return to office as the Delta...,NEGATIVE,0.999465
2,4,Can Apple’s AirTags Find Lost Pets?,We look at the pros and cons of using Apple’s ...,2021-07-20 16:06:15,https://www.nytimes.com/2021/07/20/technology/...,Technology,The New York Times,2021,7,6,20,Can Apple’s AirTags Find Lost Pets?. We look a...,POSITIVE,0.992870
3,5,Biden to Name a Critic of Big Tech as the Top ...,"Jonathan Kanter, a longtime antitrust lawyer, ...",2021-07-20 18:04:21,https://www.nytimes.com/2021/07/20/business/ka...,Business,The New York Times,2021,7,13,22,Biden to Name a Critic of Big Tech as the Top ...,POSITIVE,0.985043
4,6,The F.T.C. votes to use its leverage to make i...,An advocacy group for technology companies cri...,2021-07-22 02:02:59,https://www.nytimes.com/2021/07/21/us/politics...,U.S.,The New York Times,2021,7,17,20,The F.T.C. votes to use its leverage to make i...,NEGATIVE,0.997571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595,4316,Tariffs Begin to Leave an Imprint on the Economy,"Also, Republicans in Congress followed Trump’s...",2025-07-15 21:41:05,https://www.nytimes.com/2025/07/15/briefing/ta...,Briefing,The New York Times,2025,7,9,17,Tariffs Begin to Leave an Imprint on the Econo...,NEGATIVE,0.990673
2596,4318,"How to Keep Love Alive, With Rob Delaney of ‘D...",The Emmy-nominated actor talks about his role ...,2025-07-16 09:00:29,https://www.nytimes.com/2025/07/16/podcasts/ro...,Podcasts,The New York Times,2025,7,12,22,"How to Keep Love Alive, With Rob Delaney of ‘D...",POSITIVE,0.998707
2597,4319,Project 2025’s Other Project,Inside the plan from the Heritage Foundation t...,2025-07-16 10:00:10,https://www.nytimes.com/2025/07/16/podcasts/th...,Podcasts,The New York Times,2025,7,4,16,Project 2025’s Other Project. Inside the plan ...,NEGATIVE,0.991663
2598,4320,"Tariffs Push Prices Up, and the Supreme Court’...","Plus, why that e-book cost your library $50.",2025-07-16 10:00:12,https://www.nytimes.com/2025/07/16/podcasts/th...,Podcasts,The New York Times,2025,7,10,8,"Tariffs Push Prices Up, and the Supreme Court’...",NEGATIVE,0.997372


In [3]:
news_df.columns

Index(['Unnamed: 0', 'title', 'snippet', 'pub_date', 'web_url', 'section',
       'source', 'year', 'month', 'title_word_count', 'snippet_word_count',
       'combined_text', 'sentiment_label', 'sentiment_score'],
      dtype='object')

In [4]:
news_df['pub_date'] = pd.to_datetime(news_df['pub_date'])
news_df['Date'] = news_df['pub_date'].dt.date
news_df['Date'] = pd.to_datetime(news_df['Date'])

# Group by Date
daily_sentiment = news_df.groupby('Date').agg({
    'sentiment_score': ['mean', 'count'],
    'sentiment_label': lambda x: x.mode()[0] if not x.mode().empty else 'neutral'
}).reset_index()

# Flatten column names
daily_sentiment.columns = ['Date', 'avg_sentiment_score', 'news_count', 'dominant_sentiment']
daily_sentiment.dtypes


Date                   datetime64[ns]
avg_sentiment_score           float64
news_count                      int64
dominant_sentiment             object
dtype: object

In [5]:
daily_sentiment

,Date,avg_sentiment_score,news_count,dominant_sentiment
0,2021-07-20,0.994026,4,POSITIVE
1,2021-07-22,0.997748,5,NEGATIVE
2,2021-07-23,0.996606,3,NEGATIVE
3,2021-07-24,0.997136,1,NEGATIVE
4,2021-07-26,0.992176,3,POSITIVE
...,...,...,...,...
1052,2025-07-11,0.945150,4,NEGATIVE
1053,2025-07-12,0.993416,1,POSITIVE
1054,2025-07-14,0.980117,2,NEGATIVE
1055,2025-07-15,0.989851,3,NEGATIVE


In [6]:
stock_df = pd.read_csv('../data/stock_data.csv', skiprows=2)
stock_df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
stock_df['Date'] = pd.to_datetime(stock_df['Date'])
stock_df

,Date,Close,High,Low,Open,Volume
0,2021-07-19,139.404251,140.989624,138.640930,140.676459,121434600
1,2021-07-20,143.025162,143.954862,139.903380,140.392689,96350000
2,2021-07-21,142.291168,143.005571,141.537642,142.418393,74993500
3,2021-07-22,143.661285,145.031346,142.692447,142.819672,77338200
4,2021-07-23,145.383606,145.540189,143.778672,144.395206,71447400
...,...,...,...,...,...,...
999,2025-07-11,211.160004,212.130005,209.860001,210.570007,39765800
1000,2025-07-14,208.619995,210.910004,207.539993,209.929993,38840100
1001,2025-07-15,209.110001,211.889999,208.919998,209.220001,42296300
1002,2025-07-16,210.160004,212.399994,208.639999,210.300003,47490500


In [7]:
merged_df = pd.merge(stock_df, daily_sentiment, on='Date', how='left')

merged_df[['avg_sentiment_score', 'news_count']] = merged_df[['avg_sentiment_score', 'news_count']].fillna(0)
merged_df['dominant_sentiment'] = merged_df['dominant_sentiment'].fillna('NEUTRAL')

# merged_df['sentiment_numeric'] = merged_df['dominant_sentiment'].map({'POSITIVE': 1, 'NEUTRAL': 0, 'NEGATIVE': -1})
merged_df = merged_df.drop(columns=['dominant_sentiment'])
merged_df = merged_df.sort_values('Date').reset_index(drop=True)

merged_df

,Date,Close,High,Low,Open,Volume,avg_sentiment_score,news_count
0,2021-07-19,139.404251,140.989624,138.640930,140.676459,121434600,0.000000,0.0
1,2021-07-20,143.025162,143.954862,139.903380,140.392689,96350000,0.994026,4.0
2,2021-07-21,142.291168,143.005571,141.537642,142.418393,74993500,0.000000,0.0
3,2021-07-22,143.661285,145.031346,142.692447,142.819672,77338200,0.997748,5.0
4,2021-07-23,145.383606,145.540189,143.778672,144.395206,71447400,0.996606,3.0
...,...,...,...,...,...,...,...,...
999,2025-07-11,211.160004,212.130005,209.860001,210.570007,39765800,0.945150,4.0
1000,2025-07-14,208.619995,210.910004,207.539993,209.929993,38840100,0.980117,2.0
1001,2025-07-15,209.110001,211.889999,208.919998,209.220001,42296300,0.989851,3.0
1002,2025-07-16,210.160004,212.399994,208.639999,210.300003,47490500,0.996618,4.0


In [8]:
merged_df.dtypes

Date                   datetime64[ns]
Close                         float64
High                          float64
Low                           float64
Open                          float64
Volume                          int64
avg_sentiment_score           float64
news_count                    float64
dtype: object

In [10]:
merged_df.to_csv('../data/final_merged.csv', index=False)